In [6]:
import os 

In [1]:
import numpy as np
from copy import deepcopy

In [2]:
def gen(n, m, max_w, k=None):
    
    if k > m:
        return None
    
    for _ in range(100):
        cost_matrix = np.random.randint(0, max_w + 1, (n, m))

        types = {}
        num_t = 0
        for i in range(m):
            type_ = tuple(cost_matrix[:, i])
            if type_ in types:
                types[type_] += 1
            else:
                types[type_] = 1
                num_t += 1

        if k:
            if num_t == k:
                return types
            elif num_t > k:
                inds = np.random.choice(np.arange(num_t), k, replace=False)
                inds = [list(types.keys())[i] for i in inds]
                sum_num = 0
                
                for i in inds:
                    sum_num += types[i]
                
                res_types = {i: types[i] for i in inds}
                addition = np.random.choice(np.arange(k), m - sum_num, replace=True)
                
                for i in addition:
                    res_types[inds[i]] += 1
            
                return res_types
            else:
                continue
        else:
            return types
    
    return None

In [3]:
print(gen(2, 10, 50, 7))

{(37, 38): 3, (28, 8): 1, (1, 28): 2, (21, 40): 1, (28, 21): 1, (7, 5): 1, (28, 37): 1}


In [88]:
def print_list(f, list_):
    f.write(f"{list_[0]}")
    for i in list_[1:]:
        f.write(f" {i}")
    f.write('\n')

def create_ilp_2_players(max_w, res_types):
    types = list(res_types.keys())
    k = len(types)
    
    ms = [res_types[type_] for type_ in types]
    
    variable_number = k + 4 * k #number of types, number of "if" variables
    constraints_number = k + 6 * k + 2 * k #type number restrictions, "ifs", resulting constraints
    
    
    f = open("constraints_ilp", "w")
    f.write(f"{constraints_number} {variable_number + 1}\n")
    
    #type number restrictions
    
    for i in range(k):
        inds = np.zeros(variable_number + 1, dtype='int64')
        inds[0] = ms[i]
        inds[i + 1] = -1
        print_list(f, inds)
    
    #ifs
    linear_constraints = []
    
    #first player constraint 
    for i in range(k):
        #l : x_i \in [0, m_i - 1]
        #r: x_i = m_i
        # l + r = 1
        lin_ind = k + 3 * i + 1 
        linear_constraints.append(lin_ind)
        var_ind = k + 1 + 2 * i # l variable index
        inds = np.zeros(variable_number + 1, dtype='int64')
        inds[0] = 1
        inds[var_ind] = -1
        inds[var_ind + 1] = -1
        print_list(f, inds)
        
        # x_i - m_i <= -l
        inds = np.zeros(variable_number + 1, dtype='int64')
        inds[0] = ms[i]
        inds[i + 1] = -1
        inds[var_ind] = -1
        print_list(f, inds)
        
        # x_i - m_i >= -m_i * l
        inds = np.zeros(variable_number + 1, dtype='int64')
        inds[0] = -ms[i]
        inds[i + 1] = 1
        inds[var_ind] = ms[i]
        print_list(f, inds)
    
    
    #second player constraint 
    for i in range(k):
        #l : x_i = 0
        #r: x_i > 0
        # l + r = 1
        lin_ind = 4 * k + 3 * i + 1 
        linear_constraints.append(lin_ind)
        var_ind = 3 * k + 1 + 2 * i # l variable index
        inds = np.zeros(variable_number + 1, dtype='int64')
        inds[0] = 1
        inds[var_ind] = -1
        inds[var_ind + 1] = -1
        print_list(f, inds)
        
        # x_i >= r
        inds = np.zeros(variable_number + 1, dtype='int64')
        inds[i + 1] = 1
        inds[var_ind + 1] = -1
        print_list(f, inds)
        
        # x_i <= m_i * r
        inds = np.zeros(variable_number + 1, dtype='int64')
        inds[i + 1] = -1
        inds[var_ind + 1] = ms[i]
        print_list(f, inds)
    
    #resulting constraints
    
    a_constr_const = np.sum([types[i][0] * ms[i] for i in range(k)])
    b_constr_const = np.sum([types[i][1] * ms[i] for i in range(k)])
    
    #first player
    for i in range(k):
        inds = np.zeros(variable_number + 1, dtype='int64')
        inds[0] = -a_constr_const
        l_ind = k + 1 + 2 * i
        
        for j in range(k):
            inds[j + 1] = 2 * types[j][0]
            
        inds[l_ind] = types[i][0]
        inds[l_ind + 1] = max_w * k
        print_list(f, inds)
        
    #second player    
    for i in range(k):
        inds = np.zeros(variable_number + 1, dtype='int64')
        inds[0] = b_constr_const
        l_ind = 3 * k + 1 + 2 * i
        
        for j in range(k):
            inds[j + 1] = -2 * types[j][1]
            
        inds[l_ind] = max_w * k
        inds[l_ind + 1] = types[i][1]
        print_list(f, inds)    
    
    
    #linearity
    f.write(f"linearity {len(linear_constraints)}") 
    for i in linear_constraints:
        f.write(f" {i}")
    f.write('\n')
    
    #nonnegativity
    f.write(f"nonnegative {variable_number}") 
    for i in range(1, variable_number + 1):
        f.write(f" {i}")
    f.write('\n')
         
    f.close()
    
    
res_types = gen(2, 10000, 10, 2)


create_ilp_2_players(50, res_types)

In [89]:
print(res_types)

{(1, 8): 4908, (8, 4): 5092}


In [90]:
stream = os.popen('latte-integrale-1.7.4/latte-int-1.7.4/code/latte/count constraints_ilp')

_ = stream.readlines()

with open('numOfLatticePoints') as f:
    print(f.read())

This is LattE integrale 1.7.4
Available from http://www.math.ucdavis.edu/~latte/

Invocation: latte-integrale-1.7.4/latte-int-1.7.4/code/latte/count constraints_ilp 
Checking whether the input polytope is empty or not...size = 32 x 11
Number Type = rational
done.
Removing redundant inequalities and finding hidden equalities using cddlib...done. 
Ax <= b, given as (b|-A):
[-4908 1 0 4908 0 0 0 0 0 0 0]
[5092 0 -1 0 0 -1 0 0 0 0 0]
[-5092 0 1 0 0 5092 0 0 0 0 0]
[0 1 0 0 0 0 0 0 -1 0 0]
[0 -1 0 0 0 0 0 0 4908 0 0]
[0 0 -1 0 0 0 0 0 0 0 5092]
[-45644 2 16 1 100 0 0 0 0 0 0]
[-45644 2 16 0 0 8 100 0 0 0 0]
[59632 -16 -8 0 0 0 0 100 8 0 0]
[59632 -16 -8 0 0 0 0 0 0 100 4]
[0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0]

Ax = b, given as (b|-A):
[1 0 0 -1 -1 0 0 0 0 0 0]
[1 0 0 0 0 -1 -1 0 0 0 0]
[1 0 0 0 0 0 0 -1 -1 0 0]
[1 0 0 0 0 0 0 0 0 -1 -1]

Computing hermitean normal form.
sol:
[1 1 1 1 0 0 0 0 0 0]
Particular solution:
Basis:
New inequ

4253473



328000 unimodular cones done.
76 vertex cones done. 

Total Unimodular Cones: 328404
Maximum number of simplicial cones in memory at once: 33

****  The number of lattice points is:   ****
Total time: 84.84 sec
